In [1]:
import os, sys
import numpy as np
import torch
import torch.nn as nn
from torch import optim
from torch.nn import utils
import torch.nn.functional as F

import math, collections, itertools, random
import matplotlib.pyplot as plt
from tqdm import tqdm
from time import sleep
from livelossplot import PlotLosses  

from models.autoencoder import AE
from models.matrix_factorization import PQ
from utils.regularizers import FairnessLoss

from preprocessing.synthetic import data_loader_synthetic
from preprocessing.movielens import data_loader_movielens
from preprocessing.lastfm import data_loader_lastfm

from train import train_PQ, train_AE
from utils.metrics import metrics

%matplotlib inline

In [5]:
# 1. Choose data: synthetic, movielens, lastfm
p0, p1 = 0.4, 0.1
q0, q1 = 0.2, 0.2

data_tuple = (data_loader_synthetic(p0, p1, q0, q1)) # return ((train_data, test_data), user attribute, item attribute)
# data_tuple = (data_loader_movielens())
# data_tuple = (data_loader_lastfm())

# 2. Choose rating system: 'binary' or 'five-stars'
data_type='binary'

# 3. Choose model type: 'PQ' or 'AE'
model_type='PQ'

# 4. Choose algorithm: 'unfair', 'ours', 'VAL', 'UGF', 'CVS'
algorithm_type = 'ours'

In [6]:
# parameter settings
learning_rate = 1e-3
l_value = 0
num_epochs = 1000
lambda_ = 0.99
tau=0
n, m = data_tuple[0][0].shape[0], data_tuple[0][0].shape[1]
r = 20
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

if algorithm_type == 'unfair':
    f_criterion = None 
else: 
    f_criterion = FairnessLoss(h=0.01, tau=tau, delta=0.01, device=device, data_tuple=data_tuple, type_=algorithm_type)

# train the model
if model_type == 'PQ':
    model = PQ(data_type, n, m, 20).to(device)
    optimizer = optim.Adam(model.parameters(), lr = learning_rate)
    logs = train_PQ(data_tuple, model, optimizer, num_epochs, device, 
                    l_value=l_value, lambda_=lambda_, f_criterion=f_criterion, tau=tau)
elif model_type == 'AE':
    model = AE(data_type, n).to(device)
    optimizer = optim.Adam(model.parameters(), lr = learning_rate)
    logs = train_AE(data_tuple, model, optimizer, num_epochs, device, 
                    l_value=l_value, lambda_=lambda_, f_criterion=f_criterion, tau=tau)


In [7]:
# check the results
metrics(model,data_tuple,device, model_type, tau)

{'RMSE': 0.6977728847578246,
 'DEE': 0.02616666666666667,
 'VAL': 0.15848381774778167,
 'UGF': 0.00011666666666676484,
 'CVS': 0.0005166666666667208}